## Part 1: Pulse Rate Algorithm

### Contents
Fill out this notebook as part of your final project submission.

**You will have to complete both the Code and Project Write-up sections.**
- The [Code](#Code) is where you will write a **pulse rate algorithm** and already includes the starter code.
   - Imports - These are the imports needed for Part 1 of the final project. 
     - [glob](https://docs.python.org/3/library/glob.html)
     - [numpy](https://numpy.org/)
     - [scipy](https://www.scipy.org/)
- The [Project Write-up](#Project-Write-up) to describe why you wrote the algorithm for the specific case.


### Dataset
You will be using the **Troika**[1] dataset to build your algorithm. Find the dataset under `datasets/troika/training_data`. The `README` in that folder will tell you how to interpret the data. The starter code contains a function to help load these files.

1. Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link

-----

### Code

In [41]:
import glob
from tqdm import tqdm
import numpy as np
import scipy as sp
import scipy.io
import osPre
import scipy.signal
import os.path
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, LeaveOneGroupOut
from sklearn.metrics import mean_squared_error
def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]


def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, conFs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        conFs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    conFs = np.hstack(conFs)
    return AggregateErrorMetric(errs, conFs)

def RunPulseRateAlgorithm(data_fl, ref_fl):
    
    """
    Run the algorithm 
    
    Args: 
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
    

    Returns:
        np.array(error): Array with error for predictions
        np.array(confidence): Array with confidence for predictions
    """
    
    Fs = 125 # Sample Frequency
    window_len = 8 # Window to calculate PR
    window_shift = 2  # Difference between windows  
       
    reg, scores = Regressor()
    targets, features, sigs, subs = Data_window8(data_fl, ref_fl)
    error, confidence = [], []
    for i,feature in enumerate(features):
        est = reg.predict(np.reshape(feature, (1, -1)))[0]
        
        ppg, accx, accy, accz = sigs[i]
        
        ppg = Filter(ppg)        
        accx = Filter(accx)
        accy = Filter(accy)
        accz = Filter(accz)        
        
        n = len(ppg) * 3
        freq = np.fft.rfftfreq(n, 1/Fs)
        fft = np.abs(np.fft.rfft(ppg,n))
        fft[freq <= 40/60.0] = 0.0
        fft[freq >= 240/60.0] = 0.0
    
        est_Fs = est / 55.0
        Fs_win = 30  / 60.0
        Fs_win_e = (freq >= est_Fs - Fs_win) & (freq <= est_Fs +Fs_win)
        conf = np.sum(fft[Fs_win_e])/np.sum(fft)
        
        error.append(np.abs((est-targets[i])))
        confidence.append(conf)
    return np.array(error), np.array(confidence)

def Data_window8(data_fl, ref_fl):
    
    """
    Load and prepare the data, based on windows length and shift and filters 
    
    Args: 
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
    

    Returns:
        np.array(targets): Array with targets
        np.array(features): Array with features
        sigs: treated signals
        subs: name of file (or subject)
    """
  
    Fs=125  # Sampling frequency
    window_len = 6 # Window to calculate PR
    window_shift = 2 # Difference between windows
    
    sig = LoadTroikaDataFile(data_fl)
    ref = scipy.io.loadmat(ref_fl)["BPM0"]
    ref = np.array([x[0] for x in ref])
    subject_name = os.path.basename(data_fl).split('.')[0]        
    start_indxs, end_indxs = Indexator(sig.shape[1], len(ref), Fs, window_len,window_shift)
    targets, features, sigs, subs = [], [], [], []
    for i, s in enumerate(start_indxs):
        start_i =  start_indxs[i]
        end_i = end_indxs[i]

        ppg = sig[0, start_i:end_i]            
        accx = sig[1, start_i:end_i]
        accy = sig[2, start_i:end_i]
        accz = sig[3, start_i:end_i]

        ppg = Filter(ppg)
        accx = Filter(accx)
        accy = Filter(accy)
        accz = Filter(accz)

        feature, ppg, accx, accy, accz = CreateFeature(ppg, accx, accy, accz)

        sigs.append([ppg, accx, accy, accz])
        targets.append(ref[i])
        features.append(feature)
        subs.append(subject_name)
        
    return (np.array(targets), np.array(features), sigs, subs)

def Data_window6():
    
    """
    Load and prepare the data, based on windows length and shift and filters 
    
    Args: 
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
    

    Returns:
        np.array(targets): Array with targets
        np.array(features): Array with features
        sigs: treated signals
        subs: name of file (or subject)
    """
    
    Fs=125 # Sampling rate    
    window_len = 6 # Window to calculate PR
    window_shift = 2 # Difference between windows
    
    data_fls, ref_fls = LoadTroikaDataset()
    pbar = tqdm(list(zip(data_fls, ref_fls)), desc="Prepare Data")
    targets, features, sigs, subs = [], [], [], []
    for data_fl, ref_fl in pbar:
        sig = LoadTroikaDataFile(data_fl)
        ref = scipy.io.loadmat(ref_fl)["BPM0"]
        ref = np.array([x[0] for x in ref])
        subject_name = os.path.basename(data_fl).split('.')[0]        
        start_indxs, end_indxs = Indexator(sig.shape[1], len(ref), Fs, window_len,window_shift)
        for i, s in enumerate(start_indxs):
            start_i =  start_indxs[i]
            end_i = end_indxs[i]

            ppg = sig[0, start_i:end_i]            
            accx = sig[1, start_i:end_i]
            accy = sig[2, start_i:end_i]
            accz = sig[3, start_i:end_i]


            ppg = Filter(ppg)
            accx = Filter(accx)
            accy = Filter(accy)
            accz = Filter(accz)

            feature, ppg, accx, accy, accz = CreateFeature(ppg, accx, accy, accz)

            sigs.append([ppg, accx, accy, accz])
            targets.append(ref[i])
            features.append(feature)
            subs.append(subject_name)
        
    return (np.array(targets), np.array(features), sigs, subs)

def CreateFeature(ppg, accx, accy, accz):
    """ Create features 
    
        Args: 
        ppg, accx, accy, accz: signals
    

    Returns:
        np.array([ppg_feature, acc_feature]): features from PPG and ACC signals
        ppg, accx, accy, accz: signals
        
    """
    ppg = Filter(ppg)
    accx = Filter(accx)
    accy = Filter(accy)
    accz = Filter(accz)
    
    
    Fs = 125
    n = len(ppg) * 4
    freq = np.fft.rfftfreq(n, 1/Fs)
    fft = np.abs(np.fft.rfft(ppg,n))
    fft[freq <= 40/60.0] = 0.0
    fft[freq >= 240/60.0] = 0.0
    
    acct = np.sqrt(accx**2 + accy**2 + accz**2) # Total signal of acc
    
    acc_fft = np.abs(np.fft.rfft(acct, n))
    acc_fft[freq <= 40/60.0] = 0.0
    acc_fft[freq >= 240/60.0] = 0.0
    
    ppg_feature = freq[np.argmax(fft)]
    acc_feature = freq[np.argmax(acc_fft)]
    
    return (np.array([ppg_feature, acc_feature]), ppg, accx, accy, accz)

def RegressionAlg(features, targets, subs):
    """ 
    
    Create the regression model
    
        Args: 
        features: Features obtained from Data_window()
        targets: Targets obtained from Data_window()
        subs: individuals 
    

    Returns:
        regression: result from regression
        score: scores from the regression
        
   
    """
    AdaBoostRegressor
    regression = RandomForestRegressor(n_estimators=400,max_depth=16)
    scores = []
    lf = KFold(n_splits=5)
    splits = lf.split(features,targets,subs)
    for i, (train_idx, test_idx) in enumerate(splits):
        X_train, y_train = features[train_idx], targets[train_idx]
        X_test, y_test = features[test_idx], targets[test_idx]
        regression.fit(X_train, y_train)
        y_pred = regression.Predict(X_test)
        score = Error(y_test, y_pred)
        scores.append(score)
        
    return (regression, scores)

def Filter(signal):
    
     """ 
    
    Bandpass filter between 40 and 240 BPM
    
        Args: 
        signal: signal to be filtered
    

    Returns:
        signal:filtered signal
        b,a: components from filter
        
   
    """
    
    pass_band=(40/60.0, 240/60.0)
    Fs = 125
    b, a = scipy.signal.butter(3, pass_band, btype='bandpass', fs=Fs)
    return scipy.signal.filtfilt(b, a, signal)

def Indexator(sig_len, ref_len, Fs=125, window_len_s=10, window_shift_s=2):
    """
    Find start and end index to iterate over a set of signals
    
    Args: 
        sig_len, ref_len: signal and reference lenght
        Fs, window_len_s, window_shift: sample frequency, window lenght and window shift
    

    Returns:
        start_indxs: start index
        end_indxs: end index
    """
    # Set the length of the biggest signal with regards to the reference signal
    if ref_len < sig_len:
        n = ref_len
    else:
        n = sig_len
    
    # Start Indexes 
    start_indxs = (np.cumsum(np.ones(n) * Fs * window_shift_s) - Fs * window_shift_s).astype(int)
    
    # End Indexes (same size as the start indexes array)
    end_indxs = start_indxs + window_len_s * Fs
    return (start_indxs, end_indxs)

def Predict(reg,feature, ppg, accx, accy, accz):
    """
    Create the prediction based on the regression algorithm
    """
    est = reg.predict(np.reshape(feature, (1, -1)))[0]
    
def Error(y_test, y_pred):
    """
    Calculate error score of the prediction
    """
    return mean_squared_error(y_test, y_pred)


def Regressor():
    """
    Apply regression
    """
    fname = "outfile.npy"
    reg, scores = [], []
    if os.path.isfile(fname):
        [reg,scores] = np.load(fname,allow_pickle=True)
        #
    else:
        targets, features, sigs, subs = Data_window6()
        reg, scores = RegressionAlg(features, targets, subs)
        np.save("outfile", [reg,scores])
    return reg, scores

In [42]:
Evaluate()

9.6607033250413767

-----
### Project Write-up

Answer the following prompts to demonstrate understanding of the algorithm you wrote for this specific context.

> - **Code Description** - Include details so someone unfamiliar with your project will know how to run your code and use your algorithm. 
> - **Data Description** - Describe the dataset that was used to train and test the algorithm. Include its short-comings and what data would be required to build a more complete dataset.
> - **Algorithhm Description** will include the following:
>   - how the algorithm works
>   - the specific aspects of the physiology that it takes advantage of
>   - a describtion of the algorithm outputs
>   - caveats on algorithm outputs 
>   - common failure modes
> - **Algorithm Performance** - Detail how performance was computed (eg. using cross-validation or train-test split) and what metrics were optimized for. Include error metrics that would be relevant to users of your algorithm. Caveat your performance numbers by acknowledging how generalizable they may or may not be on different datasets.

Your write-up goes here...

**Code description**

The algorithm that predicts heart rate in BPM is based on the estimation of PPG signals and ACC. As requested by Udacity, I used only the second PPG signal (signals were measured on both wrists). The available code returns the mean absolute error and confidence. 


**Data description**

Data Description The Troika data set is used to build the algorithm (https://arxiv.org/pdf/1409.5181.pdf). We had the following signals to work on:
- ECG signal 
- PPG two signals from each wrist 
- Three channels fro the accelerometer each one corresponding to (x,y and z).
- Data is sampled at 125Hz.

**How the algorithm works**

The algorithm uses RandomForestRegressor to fit the heart rate training data.

- The specific aspects of the physiology that it takes advantage of:

Two PPG signals were used (from each wrist). As explained in the course, the capillaries in the wrist fill with blood when the heart's ventricles contract. When the blood returns to the heart there are fewer blood cells in the wrist. The PPG sensor emits a green light which can be absorbed by the red blood cells and the photodetector will see the various levels of blood flow in the reflected light. When the blood returns to the heart fewer blood cells can absorb the green light and the photo detector can see an increase in the reflected light.

- A description of the algorithm outputs:

The algorithm returns BPM and the confidence rate of this prediction. The higher the confidence rate, the higher we can trust the prediction. 

- Caveats on algorithm outputs

The confidence rate is calculated based on the magnitude of a small area that contains the estimated spectral frequency relative to the sum magnitude of the entire spectrum.

- Common failure modes 
PPG might pick higher frequencies related to motion. To deal with this problem, we take into consideration the accelerometer values.



**Algorithm Performance**

The mean absolute error was calculated and the ground truth reference signal was obtained from the ECG sensors. For cross validation I used the KFold. The error rate was around ~8 BPM on the test set. Since the data is using only limited subjects the algorithm may not be able to generalize well.